# 6.882 HW 4.2 Starter Code

See the problem set handout for instructions and deliverables.

See HW1.1 Starter Code for dependency installation instructions.

In [ ]:
# Install dependencies (run this once ever 12 hours)
!git clone https://github.com/MIT-6-882/HW utils

In [ ]:
from utils.planning_utils import AStar
import abc
import functools
import numpy as np
import time
from collections import defaultdict
from tabulate import tabulate

## Utilities

In [ ]:
class CategoricalDist:
    """A hashable categorical distribution.

    Examples:
    >>> dist = CategoricalDist({"a" : 0.3, "b" : 0.7})
    >>> print(dist["a"])
    >>> print(dist.items())
    >>> print(dist.sample(np.random))

    Parameters
    ----------
    cat_to_prob : {hashable : float}
        Maps a category to a probability. Needs to sum to 1.
    """
    def __init__(self, cat_to_prob):
        assert abs(sum(cat_to_prob.values()) - 1.) < 1e-6
        self._dict = {k : p for k, p in cat_to_prob.items() if p > 0}
        self._hashable = tuple(sorted(self._dict.items()))
        self._hash = hash(self._hashable)
        self._str = "\n".join([f"{k}:{v}" for k,v in self._hashable])

    def __hash__(self):
        return self._hash

    def __eq__(self, other):
        return self._hashable == other._hashable

    def __str__(self):
        return self._str

    def __repr__(self):
        return self._str

    def __iter__(self):
        return iter(self._dict)

    def __getitem__(self, key):
        try:
            return self._dict[key]
        except KeyError:
            return 0.

    def items(self):
        return self._dict.items()

    def sample(self, rng):
        choices, probs = zip(*self._dict.items())
        choice_idx = rng.choice(len(choices), p=probs)
        return choices[choice_idx]

    @functools.lru_cache(maxsize=None)
    def entropy(self):
        H = sum(-p * np.log(p) for p in self._dict.values())
        return H
    

@functools.lru_cache(maxsize=None)
def state_estimator(pomdp, belief_state, action, obs):
    """Get a new belief state from the previous belief state (time t),
    last action taken (time t), and the observation received (time t+1).

    Parameters
    ----------
    pomdp : POMDP
    belief_state : CategoricalDist
    action : hashable
    obs : hashable

    Returns
    -------
    next_belief_state : CategoricalDist
    """
    raise NotImplementedError("Implement me!")

## POMDPs (Environments)

In [ ]:
class POMDP:
    """An environment API that exposes transition probabilities
    and observation probabilities for use with planners.
    """
    @abc.abstractmethod
    def get_all_states(self):
        """Return a list of all possible states of the environment.

        We're restricted to small environments because we need
        this function.

        Returns
        -------
        states : [ hashable ]
        """
        raise NotImplementedError("Override me")

    @abc.abstractmethod
    def get_all_observations(self):
        """Return a list of all possible observations of the
        environment.

        We're restricted to small environments because we need
        this function.

        Returns
        -------
        observations : [ hashable ]
        """
        raise NotImplementedError("Override me")

    @abc.abstractmethod
    def get_all_actions(self):
        """Return a list of all possible actions of the environment.

        Returns
        -------
        actions : [ hashable ]
        """
        raise NotImplementedError("Override me")

    @abc.abstractmethod
    def get_transition_probabilities(self, state, action):
        """Return a dictionary of next_states to probabilities.

        Returns
        -------
        next_states : CategoricalDist
        """
        raise NotImplementedError("Override me")

    @abc.abstractmethod
    def get_observation_probabilities(self, state, action=None):
        """Return a dictionary of observations to probabilities.

        Returns
        -------
        observations : CategoricalDist
        """
        raise NotImplementedError("Override me")

    @abc.abstractmethod
    def get_reward(self, state, action):
        """Return (deterministic) reward for executing action
        in state.

        Returns
        -------
        reward : float
            Single time step reward.
        """
        raise NotImplementedError("Override me")

    @abc.abstractmethod
    def get_initial_state_distribution(self):
        """Distribution over initial states.

        Returns
        -------
        dist : CategoricalDist
        """
        raise NotImplementedError("Override me")

    @abc.abstractmethod
    def get_maximum_possible_reward(self):
        """Not part of the POMDP formalism, but included so
        we can use conversions to shortest path.

        Returns
        -------
        reward : float
            Single time step reward.
        """
        raise NotImplementedError("Override me")



class TigerPOMDP(POMDP):
    """The tiger problem discussed in class
    """
    def get_all_states(self):
        return ["tiger-left", "tiger-right"]

    def get_all_observations(self):
        # The null observation is just for the initial state
        return ["null", "hear-left", "hear-right"]

    def get_all_actions(self):
        return ["listen", "open-left", "open-right"]

    def get_initial_state_distribution(self):
        return CategoricalDist({"tiger-left" : 0.51, "tiger-right" : 0.49})

    def get_transition_probabilities(self, state, action):
        if action == "listen":
            return CategoricalDist({state : 1.0})
        # Return to beginning with random resets
        assert action in ["open-left", "open-right"]
        return self.get_initial_state_distribution()

    def get_observation_probabilities(self, state, action=None):
        # Initial or final state
        if action in [None, "open-left", "open-right"]:
            return CategoricalDist({"null" : 1.0})
        assert action == "listen", f"Unknown action {action}"
        if state == "tiger-right":
            return CategoricalDist({"hear-right" : 0.85, "hear-left" : 0.15})
        if state == "tiger-left":
            return CategoricalDist({"hear-left" : 0.85, "hear-right" : 0.15})
        raise Exception(f"Unknown state {state}")

    def get_reward(self, state, action):
        if action == "listen":
            return -1
        if action == "open-left":
            if state == "tiger-left":
                return -100
            assert state == "tiger-right"
            return 10
        if action == "open-right":
            if state == "tiger-right":
                return -100
            assert state == "tiger-left"
            return 10
        raise Exception(f"Unknown action {action}")

    def get_maximum_possible_reward(self):
        return 10


class LemonPOMDP(POMDP):
    """The 'is the car a lemon?' POMDP from class.

    Very similar to tiger!
    """
    def get_all_states(self):
        return ["lemon", "peach"]

    def get_all_observations(self):
        return ["null", "pass", "fail"]

    def get_all_actions(self):
        return ["buy", "dont-buy", "inspect"]

    def get_initial_state_distribution(self):
        return CategoricalDist({"lemon" : 0.2, "peach" : 0.8})

    def get_transition_probabilities(self, state, action):
        if action == "inspect":
            return CategoricalDist({state : 1.0})
        # Return to beginning with random resets
        assert action in ["buy", "dont-buy"]
        return self.get_initial_state_distribution()

    def get_observation_probabilities(self, state, action=None):
        # Initial or final state
        if action in [None, "buy", "dont-buy"]:
            return CategoricalDist({"null" : 1.0})
        assert action == "inspect", f"Unknown action {action}"
        if state == "lemon":
            return CategoricalDist({"pass" : 0.4, "fail" : 0.6})
        if state == "peach":
            return CategoricalDist({"pass" : 0.9, "hear-right" : 0.1})
        raise Exception(f"Unknown state {state}")

    def get_reward(self, state, action):
        if action == "inspect":
            return -9
        if action == "buy":
            if state == "lemon":
                return -100
            assert state == "peach"
            return 60
        if action == "dont-buy":
            return 0
        raise Exception(f"Unknown action {action}")

    def get_maximum_possible_reward(self):
        return 60


class SARPOMDP(POMDP):
    """Search and rescue pomdp
    """
    # Set up actions
    ALL_ACTIONS = MOVE_UP, MOVE_DOWN, MOVE_LEFT, MOVE_RIGHT, \
                  SCAN_UP, SCAN_DOWN, SCAN_LEFT, SCAN_RIGHT = range(8)

    # Set up the grid cell types
    EMPTY, WALL, FIRE, HIDDEN = range(4)

    # Set up possible scan responses (NA = not applicable)
    SCAN_RESPONSES = GOT_RESPONSE, GOT_NO_RESPONSE, NA = range(3)

    # Set up stochasticity parameters
    MOVE_NOISE_PROB = 0.1
    SCAN_NOISE_PROB = 0.1

    # Set up rewards
    TIME_REWARD = -1
    RESCUE_REWARD = 100
    FIRE_REWARD = -100

    @property
    def height(self):
        return self.GRID.shape[0]

    @property
    def width(self):
        return self.GRID.shape[1]
    
    @functools.lru_cache(maxsize=None)
    def get_all_states(self):
        """States are (robot_loc, person_loc)
        """
        states = []
        for person_loc in np.argwhere(self.GRID == self.HIDDEN):
            person_loc = tuple(person_loc)
            for possible_robot_loc in self._get_possible_robot_locs():
                states.append((possible_robot_loc, person_loc))
        return states

    @functools.lru_cache(maxsize=None)
    def get_all_observations(self):
        """Observations are (robot_loc, scan_response)
        """
        observations = []
        for scan_response in self.SCAN_RESPONSES:
            for possible_robot_loc in self._get_possible_robot_locs():
                observations.append((possible_robot_loc, scan_response))
        return observations

    @functools.lru_cache(maxsize=None)
    def get_all_actions(self):
        """Actions are moving and scanning in four directions
        """
        return self.ACTIONS

    @functools.lru_cache(maxsize=None)
    def get_initial_state_distribution(self):
        """We will always start the robot in the same initial place,
        and the distribution over rooms will be uniform
        """
        initial_robot_loc = self.INITIAL_ROBOT_LOC
        initial_states = []
        for person_loc in np.argwhere(self.GRID == self.HIDDEN):
            person_loc = tuple(person_loc)
            initial_states.append((initial_robot_loc, person_loc))
        dist = {s : 1./len(initial_states) for s in initial_states}
        return CategoricalDist(dist)

    @functools.lru_cache(maxsize=None)
    def get_transition_probabilities(self, state, action):
        """Movement effects are stochastic
        """
        # Scanning does not change the state
        if action in [self.SCAN_UP, self.SCAN_DOWN, self.SCAN_LEFT,
                      self.SCAN_RIGHT]:
            return CategoricalDist({state : 1.0})
        robot_r, robot_c = state[0]
        # Moving is deterministically null if we're in a fire
        if self.GRID[robot_r, robot_c] == self.FIRE:
            return CategoricalDist({state : 1.0})
        # Moving is stochastic if we're not in a fire
        if action == self.MOVE_UP:
            intended_delta = (-1, 0)
        elif action == self.MOVE_DOWN:
            intended_delta = (1, 0)
        elif action == self.MOVE_LEFT:
            intended_delta = (0, -1)
        elif action == self.MOVE_RIGHT:
            intended_delta = (0, 1)
        else:
            raise Exception(f"Unrecognized action {action}.")
        # Reset to the beginning if we're moving into a goal
        next_loc = (robot_r + intended_delta[0],
                    robot_c + intended_delta[1])
        if next_loc == state[1]:
            return self.get_initial_state_distribution()
        dist = defaultdict(float)
        for (dr, dc) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
            if self._is_valid_robot_loc((robot_r+dr, robot_c+dc)):
                next_robot_loc = (robot_r + dr, robot_c + dc)
            else:
                next_robot_loc = (robot_r, robot_c)
            next_state = (next_robot_loc, state[1])
            if (dr, dc) == intended_delta:
                dist[next_state] += 1.-self.MOVE_NOISE_PROB
            else:
                dist[next_state] += self.MOVE_NOISE_PROB / 3.
        return CategoricalDist(dist)

    @functools.lru_cache(maxsize=None)
    def get_observation_probabilities(self, state, action=None):
        """Always observe robot loc directly; get scan response
        stochastically when scanning
        """
        # If the action was not a scan, deterministic
        if action not in [self.SCAN_UP, self.SCAN_DOWN,
                          self.SCAN_LEFT, self.SCAN_RIGHT]:
            return CategoricalDist({(state[0], self.NA) : 1.0})
        # If the action was a scan, get a noisy response
        # First figure out the 'correct' directions to scan
        correct_scans = []
        # Delta between person and robot
        dr, dc = np.subtract(state[1], state[0])
        if dr < 0:
            correct_scans.append(self.SCAN_UP)
        elif dr > 0:
            correct_scans.append(self.SCAN_DOWN)
        if dc < 0:
            correct_scans.append(self.SCAN_LEFT)
        elif dc > 0:
            correct_scans.append(self.SCAN_RIGHT)
        assert len(correct_scans) <= 2
        if action in correct_scans:
            correct_response = self.GOT_RESPONSE
            incorrect_response = self.GOT_NO_RESPONSE
        else:
            correct_response = self.GOT_NO_RESPONSE
            incorrect_response = self.GOT_RESPONSE
        # Return a noisy response
        dist = {
            (state[0], correct_response) : 1.-self.SCAN_NOISE_PROB,
            (state[0], incorrect_response) : self.SCAN_NOISE_PROB,
        }
        return CategoricalDist(dist)

    @functools.lru_cache(maxsize=None)
    def get_reward(self, state, action):
        """Get a positive reward for rescuing the person
        """
        robot_loc, person_loc = state
        if action == self.MOVE_UP:
            intended_delta = (-1, 0)
        elif action == self.MOVE_DOWN:
            intended_delta = (1, 0)
        elif action == self.MOVE_LEFT:
            intended_delta = (0, -1)
        elif action == self.MOVE_RIGHT:
            intended_delta = (0, 1)
        else:
            return self.TIME_REWARD
        next_loc = (robot_loc[0] + intended_delta[0],
                    robot_loc[1] + intended_delta[1])
        if next_loc == person_loc:
            return self.RESCUE_REWARD
        # If in fire, big negative reward
        if 0 <= next_loc[0] < self.height and \
           0 <= next_loc[1] < self.width and \
           self.GRID[next_loc[0], next_loc[1]]:
            return self.FIRE_REWARD
        return self.TIME_REWARD

    def get_maximum_possible_reward(self):
        return self.RESCUE_REWARD

    @functools.lru_cache(maxsize=None)
    def _get_possible_robot_locs(self):
        """Helper for state and obs spaces
        """
        mask = (self.GRID == self.EMPTY) 
        mask |= (self.GRID == self.HIDDEN)
        mask |= (self.GRID == self.FIRE)
        return [tuple(loc) for loc in np.argwhere(mask)]

    @functools.lru_cache(maxsize=None)
    def _is_valid_robot_loc(self, robot_loc):
        """Helper for transitions
        """
        robot_r, robot_c = robot_loc
        if not (0 <= robot_r < self.height and \
                0 <= robot_c < self.width):
            return False
        if self.GRID[robot_r,robot_c] == self.WALL:
            return False
        assert robot_loc in self._get_possible_robot_locs()
        return True


class TinySARPOMDP(SARPOMDP):
    E, W, F, H = SARPOMDP.EMPTY, SARPOMDP.WALL, SARPOMDP.FIRE, \
                 SARPOMDP.HIDDEN
    GRID = np.array([
        [H, E, E, E, E, E, H]
    ])
    INITIAL_ROBOT_LOC = (0, 3)
    # Override actions to make this easier
    ACTIONS = [SARPOMDP.MOVE_LEFT, SARPOMDP.MOVE_RIGHT,
               SARPOMDP.SCAN_LEFT] 
    # Override noise prob to make this easier
    MOVE_NOISE_PROB = 0.0


class LargeSARPOMDP(SARPOMDP):
    E, W, F, H = SARPOMDP.EMPTY, SARPOMDP.WALL, SARPOMDP.FIRE, \
                 SARPOMDP.HIDDEN
    GRID = np.array([
        [W, W, H, E, E, E, E],
        [W, F, E, W, E, W, E],
        [F, F, E, E, E, E, H],
        [E, F, E, E, E, F, E],
        [H, E, E, E, E, F, F],
        [E, W, E, W, E, F, W],
        [E, E, E, E, H, W, W]
    ])
    INITIAL_ROBOT_LOC = (3, 3)
    ACTIONS = SARPOMDP.ALL_ACTIONS

## Approaches

In [ ]:
class POMDPApproach:
    """An agent that maintains a belief state (distribution over
    possible states) as it takes actions and receives observations.

    Parameters
    ----------
    pomdp : POMDP
        A pomdp.
    """
    def __init__(self, pomdp, **kwargs):
        self._pomdp = pomdp
        self._rng = None # set in seed
        self._belief_state = None # set in reset
        self._last_action = None # set in step
        self._states = pomdp.get_all_states()
        self._actions = pomdp.get_all_actions()

    def reset(self):
        """Tell the agent that we have started a new problem.
        """
        self._belief_state = None
        self._last_action = None

    def step(self, obs):
        """Receive an observation and produce an action to be
        immediately executed in the environment.

        Parameters
        ----------
        obs : hashable
            The observation

        Returns
        -------
        action : int
            The action is assumed to be immediately taken.
        """
        # If this is the first observation, we just need to
        # reset the belief by inverting the observation model
        if self._last_action is None:
            self._belief_state = self._update_initial_belief(obs)
        # Otherwise, we should use state_estimator to update
        # the belief state given the previous action and obs
        else:
            self._belief_state = state_estimator(self._pomdp,
                self._belief_state, self._last_action, obs)
        # Find an action
        action = self.belief_state_to_action(self._belief_state)
        self._last_action = action
        return action

    def seed(self, seed):
        """Seed the agent, just in case it's random
        """
        self._rng = np.random.RandomState(seed)

    def _update_initial_belief(self, obs):
        """Get a new initial belief given the obs

        Parameters
        ----------
        obs : hashable
            The observation

        Returns
        -------
        belief : CategoricalDist
        """
        # Pr(s | o) = Pr(o | s)Pr(s) / Pr(o)
        # Pr(o) = \sum_s' Pr(o | s')Pr(s')
        initial_state_dist = self._pomdp.get_initial_state_distribution()
        F = self._pomdp.get_observation_probabilities
        p_obs = sum([F(s)[obs] * p_s \
                     for s, p_s in initial_state_dist.items()])
        belief = {s : F(s)[obs] * p_s / p_obs \
                  for s, p_s in initial_state_dist.items()}
        return CategoricalDist(belief)

    @abc.abstractmethod
    def belief_state_to_action(self, belief_state):
        """Return an action to be immediately taken, based on the current
        belief state (self._belief_state). This is the main thing that
        differentiates subclasses.

        Parameters
        ----------
        belief_state : CategoricalDist

        Returns
        -------
        action : int
            The action to be taken immediately.
        """
        raise NotImplementedError("Override me")

### Random Actions

In [ ]:
class RandomActions(POMDPApproach):
    """Take random actions
    """
    def belief_state_to_action(self, _):
        return self._rng.choice(self._actions)

### Most Likely Observation + A*

In [ ]:
class MLOAStar(POMDPApproach):
    """Plan using the most likely observation determinization
    strategy. Do receding horizon control and replan at every
    time step.
    """
    def __init__(self, pomdp, receding_horizon=25):
        super().__init__(pomdp)
        self._receding_horizon = receding_horizon
        self._planner = self._create_planner()

    def _create_planner(self):
        """Initializes the planner that uses state estimator as
        its successor function to plan in belief space.

        Returns
        -------
        planner : AStar
        """
        # Start by creating the belief state successor function
        mlo_succ = self._create_mlo_successor_fn(self._pomdp)
        # Since we're doing finite horizon A* shortest path,
        # we need to know the current depth in the state
        successor_fn = lambda bh,a : (mlo_succ(bh[0], a), bh[1] + 1)
        # A goal is when we've reached the horizon
        check_goal_fn = lambda bh : (bh[1] >= self._receding_horizon)
        # Cost is Rmax - expected reward
        bel_rew = self._create_belief_reward_fn(self._pomdp)
        r_max = self._pomdp.get_maximum_possible_reward()
        get_cost = lambda bh,a : r_max - bel_rew(bh[0], a)
        # Get all actions
        actions = self._pomdp.get_all_actions()
        # Make the planner
        return AStar(successor_fn, check_goal_fn,
                     get_cost=get_cost, actions=actions)

    @staticmethod
    def _create_mlo_successor_fn(pomdp):
        """Set up the successor function for the planner

        Parameters
        ----------
        pomdp : POMDP

        Returns
        -------
        mlo_succ : fn
            Map from belief state and action to belief state.
        """
        def mlo_succ(b, a):
            """Most likely observation successor function.

            Given the belief state b at time t, and the action a
            taken at time t, return the belief state at time t+1.

            From lecture notes:
                successor(b, a) = SE(b, a, argmax_o P(o | b, a))

            Important note: The observation o that we are maxing
            over in that equation is the observation at time t+1,
            whereas b and a are from time t! Computing the prob
            P(o | b, a) will therefore require using the transition
            model (in addition to the observation model).

            Parameters
            ----------
            b : CategoricalDist
                The belief at time t
            a : hashable
                The action taken at time t

            Returns
            -------
            next_belief : CategoricalDist
            """
            raise NotImplementedError("Implement me!")
        return mlo_succ

    @staticmethod
    def _create_belief_reward_fn(pomdp):
        """Set up the reward function for the planner
        """
        def belief_reward(b, a):
            """The reward for a belief state is the expectation
            over rewards for the states.
            """
            r = 0.
            for s in b:
                r += b[s] * pomdp.get_reward(s, a)
            return r
        return belief_reward

    def belief_state_to_action(self, belief_state):
        # Replan on every time step!
        # Remember that with the conversion to A* shortest path,
        # we need to include the time step in the state.
        plan, _ = self._planner((belief_state, 0))
        return plan[0]

## Pipeline

In [ ]:
def run_single_test(pomdp, approach, max_horizon=50, verbose=False, rng=np.random):
    # Extract things from POMDP
    initial_state_dist = pomdp.get_initial_state_distribution()
    T = pomdp.get_transition_probabilities
    F = pomdp.get_observation_probabilities
    R = pomdp.get_reward
    # Start the evaluation
    trial_returns = 0.
    approach.reset()
    # Sample an initial state
    state = initial_state_dist.sample(rng)
    # Initialize last action to None
    action = None
    for _ in range(max_horizon):
        # Sample an observation based on current state and
        # last action taken
        obs = F(state, action).sample(rng)
        if verbose: print("obs:", obs)
        # Get action from the approach
        action = approach.step(obs)
        if verbose: print("action:", action)
        # Take action
        next_state = T(state, action).sample(rng)
        # Get reward
        trial_returns += R(state, action)
        # Update state
        state = next_state
    return trial_returns

def run_single_experiment(pomdp, approach, verbose=False, max_horizon=100,
                          num_trials=1, seed=0):
    # Initialize
    approach.seed(seed)
    rng = np.random.RandomState(seed)
    # Do testing
    test_durations = [] # seconds, one per problem
    test_returns = [] # float, cumulative rewards
    for trial in range(num_trials):
        if verbose: print(f"STARTING TRIAL {trial}")
        start_time = time.time()
        returns = run_single_test(pomdp, approach, max_horizon=max_horizon,
                                  rng=rng, verbose=verbose)
        duration = time.time() - start_time
        if verbose: print(f"TRIAL {trial} RETURNS: {returns}")
        test_durations.append(duration)
        test_returns.append(returns)
    return test_durations, test_returns

def print_results_table(env_name, results_for_env):
    print(f"\n### {env_name} ###")
    mean_table = [(a, ) + tuple(np.mean(results_for_env[a], axis=0)) \
                  for a in sorted(results_for_env)]
    columns = ["Approach", "Duration", "Returns"]
    print(tabulate(mean_table, headers=columns))


def main():
    verbose = False

    approaches = {
        "Random" : RandomActions,
        "MLO-A*" : MLOAStar,
    }

    pomdps = {
        "Tiger" : TigerPOMDP(),
        "Lemon" : LemonPOMDP(),
        "Tiny SAR" : TinySARPOMDP(),
        # "Large SAR" : LargeSARPOMDP(),
    }

    num_trials_per_env = {
        "Tiger" : 25,
        "Lemon" : 25,
        "Tiny SAR" : 10,
        "Large SAR" : 10,
    }

    receding_horizon_per_env = {
        "Tiger" : 5,
        "Lemon" : 5,
        "Tiny SAR" : 10,
        "Large SAR" : 25,
    }

    max_horizon_per_env = {
        "Tiger" : 10,
        "Lemon" : 10,
        "Tiny SAR" : 10,
        "Large SAR" : 50,
    }

    all_results = {}
    for env_name, pomdp in pomdps.items():
        results_for_env = {}
        all_results[env_name] = results_for_env
        num_trials = num_trials_per_env[env_name]
        receding_horizon = receding_horizon_per_env[env_name]
        max_horizon = max_horizon_per_env[env_name]
        for approach_name, approach_cls in approaches.items():
            approach = approach_cls(pomdp, receding_horizon=receding_horizon)
            results = run_single_experiment(pomdp, approach,
                                            max_horizon=max_horizon,
                                            num_trials=num_trials,
                                            verbose=verbose)
            results_for_env[approach_name] = list(zip(*results))

        # Print per-environment results (because of impatience)
        print_results_table(env_name, results_for_env)

    # Print final results
    print("\n" + "*" * 80)
    for env_name in pomdps:
        print_results_table(env_name, all_results[env_name])

### Fire away!

In [ ]:
main()